In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import json 
from pandas.io.json import json_normalize

df=pd.read_csv("/Users/veillonpaul-armand/Documents/Projets/Data/Carto/GN_conso_dep_level.csv")

#all closed question
questions=[{"Thème":"Fiscalité","content":{"name":"Fisca_1","question":"Afin de financer les dépenses sociales, faut-il selon vous." ,"answers":["Reculer l'âge de la retraite","Augmenter le temps de travail","Augmenter les impôts",
        "Revoir les conditions d'attribution de certaines aides sociales"],"label":["a","b","c","d"]}},
         {"Thème":"Ecologie","content":[{'name':"Ecolo_0","question":"Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?","answers":["La pollution de l'air", "Les dérèglements climatiques", "L'érosion du littoral",
         "La biodiversité et la disparition de certaines espèces"],'label':["a","b","c","d"]},
        {"name":"Ecolo_2","question":"Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_4","question":"À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_8","question":"Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_10","question":"Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle(...) ?","answers":["Oui","Non","Je n'utilise pas la voiture"],"label":["a","b","c"]},
        {"name":"Ecolo_12","question":"Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?","answers":["Les transports en commun","Le covoiturage","L'auto partage","Le transport à la demande","Le vélo"],"label":["a","b","c","d","e"]}]},
        {"Thème":"Démocratie et Citoyenneté","content":[{"name":"Cit_1","question":"En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_4","question":"Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :","answers":["Une bonne chose","Une mauvaise chose","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_7","question":"Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_10","question":"Faut-il prendre en compte le vote blanc ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_13","question":"Faut-il faciliter le déclenchement du référendum d'initiative partagée qui est applicable depuis 2015 ?","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_17","question":"Faut-il transformer le Sénat et le CESE?","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]}]},
        {"Thème":"Organisation de l'Etat","content":[{"name":"Etat_1","question":"Selon vous, l'Etat doit-il aujourd'hui transférer de nouvelles missions aux collectivités territoriales ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_3","question":"Estimez-vous avoir accès aux services publics dont vous avez besoin ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_6","question":"Avez-vous déjà utilisé certaines de ces nouvelles formes de services publics ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_7","question":"Si oui, en avez-vous été satisfait ? (nouvelles formes de services publics)","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_11","question":"Connaissez-vous le 'droit à l'erreur'?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_12","question":"Si oui, avez-vous déjà utilisé ce droit à l'erreur ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_15","question":"Faut-il donner plus d'autonomie aux fonctionnaires de terrain ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_17","question":"Faut-il revoir le fonctionnement et la formation de l'administration ?","answers":["Oui","Non"],"label":["a","b"]}]}]

          


In [12]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_TRANSITION_ECOLOGIQUE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=questions[1]["content"]


for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["DEP"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", on="DEP")
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
      

In [13]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/DEMOCRATIE_ET_CITOYENNETE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=questions[2]["content"]


for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["DEP"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", on="DEP")
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        


In [14]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=questions[3]["content"]

for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["DEP"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        all_ref=all_ref.replace([np.inf,-np.inf],np.nan).dropna()
        df=df.merge(all_ref,how="left", on="DEP")
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        


In [15]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')

question=[questions[0]["content"]]

for i in range(1):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["DEP"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        all_ref=all_ref.replace([np.inf,-np.inf],np.nan).dropna()
        df=df.merge(all_ref,how="left", on="DEP")
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   

In [18]:
df["Fisca_1a"]

0     0.308511
1     0.303571
2     0.287081
3     0.333333
4     0.294964
5     0.423145
6     0.255738
7     0.242857
8     0.258929
9     0.306220
10    0.249158
11    0.278261
12    0.321086
13    0.298947
14    0.322222
15    0.276382
16    0.353583
17    0.245192
18    0.309211
19    0.349462
20    0.349462
21    0.282000
22    0.261421
23    0.261538
24    0.286174
25    0.284182
26    0.252212
27    0.255376
28    0.296774
29    0.295615
        ...   
66    0.339286
67    0.345281
68    0.281437
69    0.372081
70    0.256198
71    0.277635
72    0.280112
73    0.319481
74    0.344828
75    0.522013
76    0.280585
77    0.276440
78    0.439037
79    0.306723
80    0.215328
81    0.263975
82    0.237569
83    0.340949
84    0.392484
85    0.276549
86    0.281369
87    0.232639
88    0.264574
89    0.302564
90    0.298969
91    0.326108
92    0.477121
93    0.265263
94    0.374364
95    0.336294
Name: Fisca_1a, Length: 96, dtype: float64